<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0


    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61
    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61


   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-19 10:09:05
-------------------
qualified stocks: 95
with latest results: 26
still star stocks: 16
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  2.11 L
Current:  1.29 C
-------------------
Today PnL: 64.11 K (0.5%)
Current PnL: -15.15 L (-10.74%)
CY Booked + Current PnL: -12.02 L (-8.53%)
-------------------
Total profit:  4.47 L
Total loss:  -19.62 L
-------------------
Total Booked + Current PnL: 14.96 L (13.11%)
Total Booked PnL: 30.11 L (26.4%)
Curr Year Booked PnL: 3.13 L (2.43%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.07 C
Est FTT PnL: 77.86 L (60.48%)
-------------------
Est LTT:  2.68 C
Est LTT PnL: 1.39 C (108.27%)
Deployed:  1.14 C
Current:  1.29 C
CAGR/XIRR %: 8.72%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
62,RELIANCE,1288.53,1526.00,6.50,X-LC,82.11,229021.0,25433.0,12092.0,-0.47,12.49,5.28,18.43,157.0,2.10,1.81,24.75,XY25,NTT,REFINERIES
69,SBILIFE,1496.49,1924.99,0.07,H-LC,73.68,148621.0,22916.0,13079.0,-0.23,18.23,8.80,28.63,147.0,1.75,1.17,32.40,AR,ATH,INSURANCE
56,PIDILITIND,2504.06,3576.00,-16.45,X-LC,41.05,92580.0,17458.0,14702.0,-0.17,23.24,15.88,42.81,120.0,1.19,0.73,16.11,X40,BTT,CHEMICALS
6,ANGELONE,2328.67,3033.00,11.80,H-SC,89.47,181844.0,30480.0,15293.0,-0.11,20.14,8.41,30.25,111.0,1.99,1.44,43.92,X40N,NTT,FINANCE
70,SBIN,760.30,863.00,-13.45,M-LC,23.16,211006.0,9526.0,17682.0,0.52,4.73,8.38,13.51,60.0,0.54,1.67,17.92,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TITAGARH,1097.23,1548.0,-3.68,H-SC,98.95,169050.0,-22965.0,101853.0,5.82,-11.96,60.25,41.08,95.0,-0.23,1.34,43.08,XY24,NTT,ENGINEERING
89,VALIANTORG,512.64,838.0,-338.93,H-SC,57.89,101478.0,-31808.0,116405.0,4.16,-23.86,114.71,63.47,161.0,-0.27,0.80,70.10,XR,NTT,CHEMICALS
84,TRIDENT,37.35,48.0,55.08,M-SC,76.84,74495.0,-17760.0,44064.0,2.65,-19.25,59.15,28.51,204.0,-0.40,0.59,25.56,XR,NTT,TEXTILES
88,VAIBHAVGBL,350.21,521.0,66.94,X-SC,88.42,111377.0,-40614.0,114741.0,2.30,-26.72,103.02,48.77,50.0,-0.35,0.88,27.93,XR,NTT,JEWELLERY
1,AARTIIND,487.04,919.0,41.26,M-SC,90.53,81087.0,-249.0,72386.0,2.30,-0.31,89.27,88.69,159.0,-0.00,0.64,36.74,XR,NTT,CHEMICALS


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
76,STARHEALTH,551.56,761.00,-2.42,M-SC,81.05,159629.0,-40587.0,116609.0,-2.47,-20.27,73.05,37.97,62.0,-0.35,1.26,28.17,XY24,NTT,INSURANCE
35,HONAUT,42646.15,58518.52,-22.02,X-MC,77.89,114165.0,-13773.0,61387.0,-1.97,-10.77,53.77,37.22,73.0,-0.22,0.90,16.56,X40N,ATH,ELECTRICAL
53,MEDANTA,1022.60,1486.00,1.55,X-MC,4.21,178980.0,23545.0,46893.0,-1.96,15.15,26.20,45.32,201.0,0.50,1.41,21.78,XY24,NTT,HEALTHCARE
41,INFY,1432.81,2275.00,-12.42,X-LC,71.58,218188.0,19027.0,98032.0,-1.27,9.55,44.93,58.78,10.0,0.19,1.72,13.91,X40,BTT,IT
46,KANSAINER,299.63,340.00,-66.39,H-SC,17.89,234315.0,-35352.0,71677.0,-1.23,-13.11,30.59,13.47,162.0,-0.49,1.85,17.28,XY24,NTT,PAINTS


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TATAMOTORS,776.02,1065.00,-51.41,M-LC,93.68,195438.0,-10207.0,86794.0,0.93,-4.96,44.41,37.24,1.0,-0.12,1.54,27.21,XY24,NTT,AUTO
38,INDIAMART,2311.97,4911.36,-51.65,H-SC,56.84,104716.0,2989.0,111386.0,1.26,2.94,106.37,112.43,2.0,0.03,0.83,23.59,AR,ATH,MISC
2,ABB,5319.37,8953.12,-35.88,H-LC,50.53,146675.0,13691.0,77151.0,0.81,10.30,52.60,68.31,4.0,0.18,1.16,19.60,AR,ATH,ELECTRICAL
82,TCS,3928.29,4998.00,-15.19,X-LC,62.11,190966.0,-21162.0,78926.0,-0.70,-9.98,41.33,27.23,5.0,-0.27,1.51,9.44,X40,BTT,IT
49,LICI,777.74,983.00,-13.28,H-LC,53.68,192640.0,18426.0,27548.0,0.35,10.58,14.30,26.39,6.0,0.67,1.52,17.94,XY25,NTT,INSURANCE


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,DMART,3484.00,5112.00,-17.43,X-LC,15.79,101678.0,18062.0,21007.0,1.12,21.60,20.66,46.73,146.0,0.86,0.80,24.46,AR,NTT,FMCG
56,PIDILITIND,2504.06,3576.00,-16.45,X-LC,41.05,92580.0,17458.0,14702.0,-0.17,23.24,15.88,42.81,120.0,1.19,0.73,16.11,X40,BTT,CHEMICALS
6,ANGELONE,2328.67,3033.00,11.80,H-SC,89.47,181844.0,30480.0,15293.0,-0.11,20.14,8.41,30.25,111.0,1.99,1.44,43.92,X40N,NTT,FINANCE
57,POLYCAB,5005.92,7541.35,9.37,H-MC,86.32,171220.0,31054.0,39946.0,0.34,22.16,23.33,50.65,55.0,0.78,1.35,30.67,X40N,ATH,CABLES
4,ADANIPORTS,1103.69,1583.00,-0.76,M-LC,85.26,192225.0,41019.0,24643.0,-0.31,27.13,12.82,43.43,37.0,1.66,1.52,33.24,XY24,NTT,MISC


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,45.26,M-SC,74.74,150622.0,13044.0,81366.0,1.35,9.48,54.02,68.62,79.0,0.16,1.19,30.58,X40N,NTT,TEXTILES
6,ANGELONE,2328.67,3033.00,11.80,H-SC,89.47,181844.0,30480.0,15293.0,-0.11,20.14,8.41,30.25,111.0,1.99,1.44,43.92,X40N,NTT,FINANCE
57,POLYCAB,5005.92,7541.35,9.37,H-MC,86.32,171220.0,31054.0,39946.0,0.34,22.16,23.33,50.65,55.0,0.78,1.35,30.67,X40N,ATH,CABLES
33,HEROMOTOCO,4311.81,6039.03,1.43,H-MC,87.37,154700.0,3787.0,56667.0,1.72,2.51,36.63,40.06,14.0,0.07,1.22,26.54,AR,ATH,AUTO
28,GILLETTE,8109.44,10330.69,-0.07,H-SC,75.79,147373.0,9513.0,28251.0,0.69,6.90,19.17,27.39,68.0,0.34,1.16,29.01,X40,ATH,FMCG


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
18,CAMS,3643.00,5250.99,-2.00,H-SC,18.95,110264.0,8260.0,36762.0,1.21,8.10,33.34,44.14,22.0,0.22,0.87,25.65,XR,ATH,MISC
33,HEROMOTOCO,4311.81,6039.03,1.43,H-MC,87.37,154700.0,3787.0,56667.0,1.72,2.51,36.63,40.06,14.0,0.07,1.22,26.54,AR,ATH,AUTO
94,WIPRO,243.46,420.00,-8.35,M-LC,61.05,157375.0,6430.0,103033.0,-0.19,4.26,65.47,72.51,52.0,0.06,1.24,18.49,XR,NTT,IT
12,BANKINDIA,113.49,190.00,-33.09,M-MC,12.63,132429.0,3277.0,83788.0,1.78,2.54,63.27,67.42,41.0,0.04,1.05,28.80,XR,NTT,BANKS
38,INDIAMART,2311.97,4911.36,-51.65,H-SC,56.84,104716.0,2989.0,111386.0,1.26,2.94,106.37,112.43,2.0,0.03,0.83,23.59,AR,ATH,MISC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
89,VALIANTORG,512.64,838.00,-338.93,H-SC,57.89,101478.0,-31808.0,116405.0,4.16,-23.86,114.71,63.47,161.0,-0.27,0.80,70.10,XR,NTT,CHEMICALS
86,UJJIVANSFB,52.77,60.00,40.02,M-SC,34.74,121419.0,-21060.0,40578.0,0.85,-14.78,33.42,13.70,214.0,-0.52,0.96,43.31,X40N,NTT,BANKS
12,BANKINDIA,113.49,190.00,-33.09,M-MC,12.63,132429.0,3277.0,83788.0,1.78,2.54,63.27,67.42,41.0,0.04,1.05,28.80,XR,NTT,BANKS
69,SBILIFE,1496.49,1924.99,0.07,H-LC,73.68,148621.0,22916.0,13079.0,-0.23,18.23,8.80,28.63,147.0,1.75,1.17,32.40,AR,ATH,INSURANCE
87,UNIONBANK,123.87,163.00,-12.58,M-LC,49.47,152199.0,11359.0,33134.0,0.63,8.07,21.77,31.59,48.0,0.34,1.20,32.21,XY24,NTT,BANKS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
91,VIPIND,488.80,718.00,-892.84,H-SC,100.00,75524.0,-19792.0,64490.0,1.59,-20.76,85.39,46.89,82.0,-0.31,0.60,48.96,X40N,NTT,MISC
83,TITAGARH,1097.23,1548.00,-3.68,H-SC,98.95,169050.0,-22965.0,101853.0,5.82,-11.96,60.25,41.08,95.0,-0.23,1.34,43.08,XY24,NTT,ENGINEERING
79,TANLA,917.30,1963.11,-38.20,H-SC,97.89,151093.0,-88322.0,361278.0,1.29,-36.89,239.11,114.01,20.0,-0.24,1.19,38.58,AR,ATH,IT
93,WHIRLPOOL,1151.42,2270.00,-34.41,M-SC,96.84,165303.0,19073.0,122985.0,0.40,13.04,74.40,97.15,27.0,0.16,1.31,41.22,X40,NTT,DURABLES
75,SONACOMS,578.05,1006.00,-27.23,M-MC,95.79,94036.0,-7123.0,82018.0,-1.01,-7.04,87.22,74.03,49.0,-0.09,0.74,32.16,AR,BTT,AUTO


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,17.19
1,25,38.83
2,50,68.19


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    34.98
X40N    15.47
X40     14.32
AR      14.13
XR      10.47
XY25     8.32
X200     1.19
SR       1.12
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    22.85
M-SC    17.96
X-LC    16.15
H-MC     9.01
M-LC     8.34
H-LC     5.95
M-MC     5.57
X-MC     5.55
X-SC     2.85
L-SC     2.29
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,11.74,0.30,31.41
BANKS,11.01,-8.17,61.23
IT,9.15,-19.36,85.95
FINANCE,7.57,-15.57,67.73
MISC,7.24,-23.86,77.16
INSURANCE,6.13,0.73,29.20
PAINTS,5.57,-16.69,51.22
AUTO,4.42,-7.93,54.98
ELECTRICAL,4.12,-6.60,60.91


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2922399.0,27
AR,1273578.0,15
XR,1218482.0,15
X40N,1204782.0,18
X40,779836.0,12
SR,203904.0,2
XY25,116343.0,5
X200,66949.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2295770.0,22
M-SC,1966108.0,21
X-LC,696187.0,12
M-MC,561902.0,5
H-MC,466265.0,8
X-SC,348760.0,4
M-LC,323866.0,6
L-SC,306455.0,4
X-MC,300299.0,5


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      884086.0      6
M-SC       XY24      741745.0      6
H-SC       AR        532179.0      3
           X40N      514863.0      7
M-SC       X40N      435561.0      6
           XR        418234.0      5
M-MC       XY24      396096.0      3
X-LC       X40       376579.0      5
H-SC       XR        279064.0      4
M-LC       XY24      203151.0      4
M-SC       X40       193493.0      2
H-LC       AR        191483.0      4
X-LC       AR        181546.0      3
H-MC       XY24      168367.0      2
L-SC       XR        161627.0      2
X-MC       XY24      145762.0      2
H-MC       AR        130123.0      2
X-SC       SR        119792.0      1
           XR        114741.0      1
           X40N      114227.0      2
M-LC       XR        103033.0      1
X-MC       X40N      100185.0      2
M-SC       AR         92963.0      1
H-SC       X40        85578.0      2
M-SC       SR         84112.0      1
M-MC       XR         83788.0      1
           AR         82018.0      1
L-SC       XY24       81562.0      1
X-LC       XY25       71113.0      3
H-MC       X40        69834.0      2
X-LC       X200       66949.0      1
L-SC       AR         63266.0      1
H-MC       XR         57995.0      1
X-MC       X40        54352.0      1
H-MC       X40N       39946.0      1
H-LC       XY25       27548.0      1
M-LC       XY25       17682.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 30.0 seconds
